# Summary of time taken and brier scores for jaxlogit, xlogit, and biogeme
Where the estimation is using draws = 600 (suboptimal but highest without running out of memory in biogeme), and training and test data is separated.

| | jaxlogit | xlogit | biogeme |
|---|---|---|---|
|Making Model | 37.7s | 16.9s | 4:15 |
|Estimating | 1.6s | 0.0s | 15.4s |
|Brier Score | 0.6345 | 0.6345 | 0.6345 |

# Setup

In [ ]:
import pandas as pd
import numpy as np
import jax
import pathlib
import xlogit
import sklearn

from jaxlogit.mixed_logit import MixedLogit, ConfigData
from jaxlogit.utils import wide_to_long

#  64bit precision
jax.config.update("jax_enable_x64", True)

# Get the full electricity dataset

Use for jaxlogit and xlogit. Adjustusting n_draws can improve accuracy, but Biogeme cannot handle 700 or more draws with this data set.

In [111]:
df = pd.read_csv(pathlib.Path.cwd() / "electricity_long.csv")
varnames = ['pf', 'cl', 'loc', 'wk', 'tod', 'seas']
n_draws = 600

Reshape the data so it can be passed to test_train_split in a wide format. Additionally, xlogit and jaxlogit require long format while biogeme requires a wide format.

In [112]:
df_long = pd.read_csv(pathlib.Path.cwd() / "electricity_long.csv")
choice_df = df_long.loc[df_long['choice'] == 1, ['id', 'chid', 'alt']]
choice_df = choice_df.rename(columns={'alt': 'choice'})
df_wide = df_long.pivot(index=['id', 'chid'], columns='alt', values=varnames)
df_wide.columns = [f'{var}_{alt}' for var, alt in df_wide.columns]
df_wide = df_wide.reset_index()
df = df_wide.merge(
    choice_df,
    on=['id', 'chid'],
    how='inner',
    validate='one_to_one'
)

df_wide_train, df_wide_test = sklearn.model_selection.train_test_split(df, train_size=0.8)
df_train = wide_to_long(df_wide_train, "chid", [1,2,3,4], "alt", varying=varnames, panels=True)
df_train = df_train.sort_values(['chid', 'alt'])
df_test = wide_to_long(df_wide_test, "chid", [1,2,3,4], "alt", varying=varnames, panels=True)
df_test = df_test.sort_values(['chid', 'alt'])

In [113]:
X_train = df_train[varnames]
y_train = df_train['choice']

ids_train = df_train['chid']
alts_train = df_train['alt']
panels_train = df_train['id']

X_test = df_test[varnames]
y_test = df_test['choice']

ids_test = df_test['chid']
alts_test = df_test['alt']
panels_test = df_test['id']

In [114]:
randvars = {'pf': 'n', 'cl': 'n', 'loc': 'n', 'wk': 'n', 'tod': 'n', 'seas': 'n'}

model_jax = MixedLogit()
model_x = xlogit.MixedLogit()

config = ConfigData(
    panels=panels_train,
    n_draws=n_draws,
    skip_std_errs=True,  # skip standard errors to speed up the example
    batch_size=None,
    optim_method="L-BFGS-B",
)
init_coeff = None

Make the model in jaxlogit

In [115]:
model_jax.fit(
    X=X_train,
    y=y_train,
    varnames=varnames,
    ids=ids_train,
    alts=alts_train,
    randvars=randvars,
    config=config
)
display(model_jax.summary())
init_coeff_j = model_jax.coeff_

    Message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
    Iterations: 84
    Function evaluations: 100
Estimation time= 32.6 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
pf                     -0.9876099     1.0000000    -0.9876099         0.323    
cl                     -0.2254359     1.0000000    -0.2254359         0.822    
loc                     2.2976453     1.0000000     2.2976453        0.0216 *  
wk                      1.5896322     1.0000000     1.5896322         0.112    
tod                    -9.6253674     1.0000000    -9.6253674      1.17e-21 ***
seas                   -9.6283633     1.0000000    -9.6283633      1.13e-21 ***
sd.pf                  -1.4346222     1.0000000    -1.4346222         0.151    
sd.cl                  -0.7809900     1.0000000    -0.7809900      

None

Make the model in xlogit

In [116]:
model_x.fit(
    X=X_train,
    y=y_train,
    varnames=varnames,
    ids=ids_train,
    alts=alts_train,
    randvars=randvars,
    panels=panels_train,
    n_draws=n_draws,
    skip_std_errs=True,  # skip standard errors to speed up the example
    batch_size=None,
    optim_method="L-BFGS-B",
)
display(model_x.summary())
init_coeff_x = model_x.coeff_

Optimization terminated successfully.
    Message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
    Iterations: 62
    Function evaluations: 71
Estimation time= 18.6 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
pf                     -0.9840832     1.0000000    -0.9840832         0.325    
cl                     -0.2279975     1.0000000    -0.2279975          0.82    
loc                     2.3256763     1.0000000     2.3256763        0.0201 *  
wk                      1.5943038     1.0000000     1.5943038         0.111    
tod                    -9.5490129     1.0000000    -9.5490129       2.4e-21 ***
seas                   -9.6210898     1.0000000    -9.6210898      1.22e-21 ***
sd.pf                  -0.2091180     1.0000000    -0.2091180         0.834    
sd.cl                   0.3692

None

Predict from the model using jaxlogit

In [ ]:
model = model_jax 
config = ConfigData(
    panels=panels_test,
    n_draws=n_draws,
    skip_std_errs=True,  # skip standard errors to speed up the example
    batch_size=None,
    optim_method="L-BFGS-B",
)
config.init_coeff = init_coeff_j

NameError: name 'deepcopy' is not defined

In [ ]:
prob_jj = model.predict(X_test, varnames, alts_test, ids_test, randvars, config)

Predict from the model using xlogit

In [ ]:
model = model_x
_, prob_xx = model.predict(X_test, varnames, alts_test, ids_test, isvars=None, panels=panels_test, n_draws=n_draws, return_proba=True)

# Biogeme

In [88]:
import biogeme.biogeme_logging as blog
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Draws, log, MonteCarlo, PanelLikelihoodTrajectory
import biogeme.database as db
from biogeme.expressions import Variable
import numpy as np
import sklearn

logger = blog.get_screen_logger()
logger.setLevel(blog.INFO)

In [89]:
database_train = db.Database('electricity', df_wide_train)
database_train.panel('id')
database_test = db.Database('electricity', df_wide_test)

Create variables and expression for utility of each alternative (V).

In [90]:
X = {
    name: {
        j: Variable(f"{name}_{j}")
        for j in [1,2,3,4]
    }
    for name in varnames
}

alt_1 = Beta('alt_1', 0, None, None, 0)
alt_2 = Beta('alt_2', 0, None, None, 0)
alt_3 = Beta('alt_3', 0, None, None, 0)
alt_4 = Beta('alt_4', 0, None, None, 1)

pf_mean = Beta('pf_mean', 0, None, None, 0)
pf_sd = Beta('pf_sd', 1, None, None, 0)
cl_mean = Beta('cl_mean', 0, None, None, 0)
cl_sd = Beta('cl_sd', 1, None, None, 0)
loc_mean = Beta('loc_mean', 0, None, None, 0)
loc_sd = Beta('loc_sd', 1, None, None, 0)
wk_mean = Beta('wk_mean', 0, None, None, 0)
wk_sd = Beta('wk_sd', 1, None, None, 0)
tod_mean = Beta('tod_mean', 0, None, None, 0)
tod_sd = Beta('tod_sd', 1, None, None, 0)
seas_mean = Beta('seas_mean', 0, None, None, 0)
seas_sd = Beta('seas_sd', 1, None, None, 0)

pf_rnd = pf_mean + pf_sd * Draws('pf_rnd', 'NORMAL')
cl_rnd = cl_mean + cl_sd * Draws('cl_rnd', 'NORMAL')
loc_rnd = loc_mean + loc_sd * Draws('loc_rnd', 'NORMAL')
wk_rnd = wk_mean + wk_sd * Draws('wk_rnd', 'NORMAL')
tod_rnd = tod_mean + tod_sd * Draws('tod_rnd', 'NORMAL')
seas_rnd = seas_mean + seas_sd * Draws('seas_rnd', 'NORMAL')

choice = Variable('choice')

V = {
    j: pf_rnd * X['pf'][j] + cl_rnd * X['cl'][j] + loc_rnd * X['loc'][j] + wk_rnd * X['wk'][j] + tod_rnd * X['tod'][j] + seas_rnd * X['seas'][j]
    for j in [1,2,3,4]
}

In [91]:
prob = models.logit(V, None, choice)
logprob = log(MonteCarlo(PanelLikelihoodTrajectory(prob)))

the_biogeme = bio.BIOGEME(
    database_train, logprob, number_of_draws=n_draws, seed=999, generate_yaml=False, generate_html=False
)
the_biogeme.model_name = 'model_b'
results = the_biogeme.estimate()
print(results)

The number of draws (600) is low. The results may not be meaningful. 
The number of draws (600) is low. The results may not be meaningful. 
The number of draws (600) is low. The results may not be meaningful. 


Results for model model_b
Nbr of parameters:		12
Sample size:			361
Observations:			3446
Excluded data:			0
Final log likelihood:		-3217.741
Akaike Information Criterion:	6459.482
Bayesian Information Criterion:	6506.149



In [92]:
P = {
    j: MonteCarlo(models.logit(V, None, j))
    for j in [1, 2, 3, 4]
}

simulate = {
    f'Prob_alt{j}': P[j]
    for j in [1, 2, 3, 4]
}

biogeme_sim = bio.BIOGEME(database_test, simulate)
biogeme_sim.model_name = 'per_choice_probs'

probs = biogeme_sim.simulate(results.get_beta_values())

# Test the results

Compare coefficients

In [105]:
print(results.get_beta_values().keys())
print(model_jax.coeff_names)

dict_keys(['pf_mean', 'pf_sd', 'cl_mean', 'cl_sd', 'loc_mean', 'loc_sd', 'wk_mean', 'wk_sd', 'tod_mean', 'tod_sd', 'seas_mean', 'seas_sd'])
['pf' 'cl' 'loc' 'wk' 'tod' 'seas' 'sd.pf' 'sd.cl' 'sd.loc' 'sd.wk'
 'sd.tod' 'sd.seas']


In [108]:
print("{:>13} {:>13} {:>13} {:>13}".format("Estimate", "Jaxlogit", "xlogit", "Biogeme"))
print("-" * 56)
fmt = "{:13} {:13.7f} {:13.7f} {:13.7f}"
coeff_names = {'pf': 'pf_mean', 'sd.pf': 'pf_sd', 'cl': 'cl_mean', 'sd.cl': 'cl_sd', 'loc': 'loc_mean', 'sd.loc': 'loc_sd', 'wk': 'wk_mean', 'sd.wk': 'wk_sd', 'tod': 'tod_mean', 'sd.tod': 'tod_sd', 'seas': 'seas_mean', 'sd.seas': 'seas_sd'}
for i in range(len(model_jax.coeff_)):
    name = model_jax.coeff_names[i]
    print(fmt.format(name[:13], 
                     model_jax.coeff_[i], 
                     model_x.coeff_[i], 
                     results.get_beta_values()[coeff_names[name]]))
print("-" * 56)

     Estimate      Jaxlogit        xlogit       Biogeme
--------------------------------------------------------
pf               -1.0067627    -1.0110497    -0.9820496
cl               -0.2320074    -0.2272688    -0.2314236
loc               2.2454414     2.2668891     2.2612401
wk                1.6752485     1.6807268     1.5909022
tod              -9.6596921    -9.6469617    -9.2714993
seas             -9.8234901    -9.8809136    -9.6652957
sd.pf            -1.3877084    -0.2261751     0.2288702
sd.cl            -0.6818270     0.4151835     0.3988945
sd.loc            1.5973204     1.7943527     1.6836334
sd.wk             0.7381698     1.1102512     1.0795784
sd.tod            2.0950597     2.3738184     2.0234659
sd.seas           1.2466002     1.5305747     1.5637653
--------------------------------------------------------


Compute the brier score:

In [30]:
# Jaxlogit
y = np.reshape(y_test, (prob_jj.shape[0], -1))
print(sklearn.metrics.brier_score_loss(y, prob_jj))

0.625050371323328


In [31]:
# xlogit
y = np.reshape(y_test, (prob_xx.shape[0], -1))
print(sklearn.metrics.brier_score_loss(y, prob_xx))

0.6254590395576656


In [32]:
# Biogeme
y = df_wide_test['choice']
print(sklearn.metrics.brier_score_loss(y, probs))

0.6258855285619304
